In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt



In [2]:
list1 = pd.read_csv('bank.csv', sep = ';' )
# X = list1.iloc[:, 0:16].values
# y = list1.iloc[:, -1].values
# list1

list1.isna()
list1.head()



,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [3]:
list1['job'].unique()  # 12 possible jobs 
list1['education'].unique()
list1['default'].unique()
list1['housing'].unique()
list1['poutcome'].unique()  # contact,month, nominal,  p_outcome ordinal

array(['unknown', 'failure', 'other', 'success'], dtype=object)

### Ordinal and Label encoding 

In [4]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
enc = OrdinalEncoder()
le = LabelEncoder()

list1.head()
# the highest rank is 0, I will now rank the job from the normally most lucrative to the least lucrative
# Jobs
job = ['entrepreneur','management', 'admin.', 'self-employed', 'blue-collar', 'technician','services', 'retired', 'housemaid', 'student', 'unemployed', 'unknown']
enc_job = OrdinalEncoder(categories=[job])
list1[['job']] = enc_job.fit_transform(list1[['job']])

# Education
education = ['tertiary', 'secondary', 'primary', 'unknown' ]
enc_edu = OrdinalEncoder( categories = [education] )
list1[['education']]= enc_edu.fit_transform(list1[['education']])

# default    
default = ['yes', 'no']
enc_def = OrdinalEncoder( categories = [default] )
list1[['default']] = enc_def.fit_transform(list1[['default']])

# Housing
housing = ['yes', 'no']
enc_def = OrdinalEncoder( categories = [default] )    # All the yes and no values shall use enc_def 
list1['housing'] = enc_def.fit_transform(list1[['housing']])

# Loan
loan = ['yes', 'no']
enc_def = OrdinalEncoder( categories = [default] )
list1['loan'] = enc_def.fit_transform(list1[['loan']])

# y (the dependent variable)
y = ['yes', 'no']
enc_def = OrdinalEncoder( categories = [default] )
list1['y'] = enc.fit_transform(list1[['y']])

# Marital 
marital = ['married', 'single', 'divorced']
enc_mar = OrdinalEncoder( categories = [marital] )
list1['marital'] = enc_mar.fit_transform(list1[['marital']])
list1.head()

# poutcome
poutcome = ['success','other','unknown','failure']
enc_p = OrdinalEncoder( categories = [poutcome] )
list1['poutcome'] = enc_p.fit_transform(list1[['poutcome']])
list1.head()

# label encoding contact and month cuz their rows are Nominally related 
lst = ['contact','month']
for i in lst:
    list1[i] = le.fit_transform(list1[i])
    


### Feature selection

In [5]:
X = list1.iloc[:, 0:16].values
y = list1.iloc[:, -1].values

In [6]:
#splitting into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Feature Scaling

In [7]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### Training and perfomance metrics

In [8]:
# Using Logistic Regression

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [9]:
# confusion matrix for logistic regression
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1159   31]
 [ 126   41]]


In [10]:
# applying k-fold cross validation for logistic regression
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print(accuracies)
# the cv = 10 arguement means that the process should cross validate 10 times, the accuracies are printed, displayed in a list form
print(accuracies.mean()) 
# mean of accuracies
print(accuracies.std())
# standard deviation of accuracies

[0.87697161 0.89589905 0.9022082  0.89274448 0.89240506 0.89873418
 0.91455696 0.90506329 0.90822785 0.89240506]
0.8979215748911873
0.009917927023586874


In [11]:
# Using Support Vector Machine

from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
#gaussian kernel used here
classifier.fit(X_train, y_train)

SVC(random_state=0)

In [12]:
# confusion matrix for kernel SVM
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
y_predl = []
# for i in y_pred:
#     if i == 0:
#         y_predl.append('Yes')
#     if i ==1:
#         y_predl.append('No')
# print(y_predl)


[[1177   13]
 [ 137   30]]


In [13]:
# applying k-fold cross validation for kernel SVM
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print(accuracies)
print(accuracies.mean()) 
# mean of accuracies
print(accuracies.std())
# standard deviation of accuracies

[0.88012618 0.9148265  0.89274448 0.89589905 0.88291139 0.89873418
 0.91455696 0.90189873 0.89556962 0.90506329]
0.8982330391726231
0.010988315730336307


In [14]:
# Using XGBClassifier

from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [15]:
# confusion matrixfor XGBClassifier
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1147   43]
 [ 104   63]]


In [16]:
# applying k-fold cross validation XGBClasssifier
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print(accuracies)
print(accuracies.mean()) 
# mean of accuracies
print(accuracies.std())
# standard deviation of accuracies

[0.87697161 0.89274448 0.89589905 0.88012618 0.89240506 0.89240506
 0.91139241 0.90189873 0.89556962 0.89873418]
0.8938146388212275
0.0093922180391568


In [17]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=10, criterion = 'entropy', random_state=0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [18]:
# confusion matrix for Random forest classifier
# y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1147   43]
 [ 104   63]]


In [19]:
# applying k-fold cross validation for Random forest classifier
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print(accuracies)
print(accuracies.mean()) 
# mean of accuracies
print(accuracies.std())
# standard deviation of accuracies

[0.88328076 0.90851735 0.9148265  0.88643533 0.91139241 0.90189873
 0.90189873 0.90189873 0.89556962 0.89873418]
0.9004452341971808
0.009569505399532508
